<a href="https://colab.research.google.com/github/faridkharis/kmeans-clustering-based-on-app-review/blob/master/2_Text_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
path = 'drive/MyDrive/Colab Notebooks/App Improvement Recommendation System/Data/'

In [4]:
df = pd.read_csv(path + 'data-ulasan-oy.csv')
df = df.drop(columns=['Unnamed: 0'])
df

,reviewId,content,score
0,1c104f86-7b0e-4efb-8610-7aa2eef5850e,"aplikasi jelek, bagusan flip",1
1,93849d50-aa43-4bea-8cb4-2b68e70bea03,Confidential,1
2,1520cf87-f7bf-4be6-886a-7c7e6d7d40ac,"Aplikasi yg tidak bermutu,,, jgn percaya sma k...",1
3,861cd396-7a64-484a-81c2-aae3c986e472,kenapa susah sekali menghubungkan IB dari BRI,1
4,bd55e03c-3cf0-4a4c-af68-79bb1883475d,"Saya menggunakan perangkat Redmi Note 5, mengg...",1
...,...,...,...
2810,2c62146d-a521-45f9-aaac-5e371f93fc92,Hallo admin per tanggal 14 agustus poin kok le...,1
2811,0bfce725-03e6-4abd-aa1f-7cb68d9a3f6a,OY! jangan Pelit2 jadi orang. Chat banyak dape...,1
2812,427e2ae7-7fc8-4bf0-bd50-5a3bda4acffe,Up,1
2813,45cf797e-27e2-4eac-be22-7c7265caad9c,Chat pagi sampek sore cuma hari ini doang dpt ...,1


# Case Folding

In [5]:
df['content'] = df['content'].str.lower()

df.head()

,reviewId,content,score
0,1c104f86-7b0e-4efb-8610-7aa2eef5850e,"aplikasi jelek, bagusan flip",1
1,93849d50-aa43-4bea-8cb4-2b68e70bea03,confidential,1
2,1520cf87-f7bf-4be6-886a-7c7e6d7d40ac,"aplikasi yg tidak bermutu,,, jgn percaya sma k...",1
3,861cd396-7a64-484a-81c2-aae3c986e472,kenapa susah sekali menghubungkan ib dari bri,1
4,bd55e03c-3cf0-4a4c-af68-79bb1883475d,"saya menggunakan perangkat redmi note 5, mengg...",1


# Tokenizing

In [6]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
import nltk
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

In [7]:
# ------ Tokenizing ---------

nltk.download('punkt')

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
df['content'] = df['content'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['content'] = df['content'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df['content'] = df['content'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df['content'] = df['content'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['content'] = df['content'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['content'] = df['content'].apply(remove_singl_char)

# NLTK word tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['content'] = df['content'].apply(word_tokenize_wrapper)

df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,reviewId,content,score
0,1c104f86-7b0e-4efb-8610-7aa2eef5850e,"[aplikasi, jelek, bagusan, flip]",1
1,93849d50-aa43-4bea-8cb4-2b68e70bea03,[confidential],1
2,1520cf87-f7bf-4be6-886a-7c7e6d7d40ac,"[aplikasi, yg, tidak, bermutu, jgn, percaya, s...",1
3,861cd396-7a64-484a-81c2-aae3c986e472,"[kenapa, susah, sekali, menghubungkan, ib, dar...",1
4,bd55e03c-3cf0-4a4c-af68-79bb1883475d,"[saya, menggunakan, perangkat, redmi, note, me...",1
...,...,...,...
2810,2c62146d-a521-45f9-aaac-5e371f93fc92,"[hallo, admin, per, tanggal, agustus, poin, ko...",1
2811,0bfce725-03e6-4abd-aa1f-7cb68d9a3f6a,"[oy, jangan, pelit, jadi, orang, chat, banyak,...",1
2812,427e2ae7-7fc8-4bf0-bd50-5a3bda4acffe,[up],1
2813,45cf797e-27e2-4eac-be22-7c7265caad9c,"[chat, pagi, sampek, sore, cuma, hari, ini, do...",1


# Stopword Removal

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend([ 'indonesia', 'kulina', 'aplikasi', 'bobobox', 'oy', 'stockbit', 'payfazz', 'fazz', 'bukuwarung', 'buku', 'warung', 'flip', 'neu', 
                       'happyfresh', 'ipiring', 'nya', 'iya', 'coba', 'sih', 'bintang', 'terima', 'kasih', 'terimakasih', 'the', 'deh', 'in', 'bi',
                       'pagi', 'dong', 'moga', 'ya', 'banget', 'jaya', 'maju', 'bagus', 'suka', 'lumayan', 'cepat', 'tambah', 'tingkat', 'layan', 
                       'ribet', 'oke', 'mantap', 'bantu', 'banget', 'pakai', 'terima', 'kasih', 'keren', 'mudah', 'manfaat', 'kecewa', 'tolong',
                       'gampang', 'sukses', 'admin', 'susah', 'turun', 'naikin', 'auto', 'nomor', 'uninstall', 'hp', 'ganti', 'jelek', 'download',
                       'bangkrut', 'capek', 'melulu', 'ya', 'hilang', 'up', 'masuk', 'hapus', 'lancar', 'pokok', 'alhamdulillah', 'biar', 'minimal', 
                       'laku', 'aman', 'jaga', 'terimakasih', 'terima kasih', 'gagal', 'gratis', 'uang', 'transaksi', 'lari', 'sungguh', 'tanggap',
                       'bikin', 'nyaman', 'sulit', 'otomatis', 'butuh', 'nyata', 'nyangkut', 'selamat', 'nasib', 'kena', 'maaf', 'pindah', 'belah',
                       'maklum', 'kaum', 'pas', 'muncul', 'tulis', 'kontol', 'salah', 'ulang', 'hasil', 'saing', 'kompetitor', 'rilis', 'muat', 'lambat',
                       'kendala', 'pending', 'sesuai', 'nominal', 'eh', 'via', 'tangan', 'sedang apa', 'gunain', 'kali', 'letak', 'ngetik', 'keyboard',
                       'eja', 'buruk', 'matang', 'mulus', 'lebih baik', 'beda', 'ayo', 'tarik', 'tinggal', 'kaya', 'turunin', 'kasi', 'langsung', 'kesini',
                       'share', 'whatsapp', 'galeri', 'paham', 'faktor', 'format', 'mohon', 'kirim', 'efektif', 'nama', 'barang', 'potong', 'angka',
                       'cantik', 'tera', 'jam', 'kadaluarsa', 'proses', 'lapor', 'sistem', 'buka', 'tutup', 'pilih', 'nomor', 'kartu', 'alih', 'utama',
                       'bayar', 'mobile', 'layar', 'mantap', 'mantul', 'terus', 'bot', 'hebat', 'kembang', 'semangat', 'keren', 'rekomemded', 'efisien',
                       'canda', 'zuztjd', 'support', 'digital', 'praktis', 'urus', 'jos', 'gandos', 'pelit', 'jamaah', 'rate', 'malu', 'sampah', 'parah',
                       'setan', 'yh', 'zzz', 'mahal', 'nyesel', 'normal', 'habis', 'kamera', 'nol', 'nom', 'masak', 'si', 'bodoh', 'mentang', 'harap', 'payah',
                       'orang', 'balikin', 'gs', 'rendem', 'bilang', 'nambah', 'manual', 'heuuu', 'henti', 'enak', 'db', 'komenan', 'aa', 'abai', 'heh',
                       'hha', 'dinaikin', 'sekaliuninstal', 'nga', 'zonk', 'hilanggg', 'tolol', 'ngumpulin', 'muter', 'hadehhh', 'nyarinya', 'malas', 'main',
                       'nerima', 'oi', 'asu', 'kendor', 'jancok', 'ora', 'kenek', 'sekrang', 'keluh', 'timbang', 'kucing', 'ku', 'copot', 'indah', 'pasang',
                       'kesel', 'sangat', 'besar', 'isi', 'tunggu'])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['content'] = df['content'].apply(stopwords_removal) 

In [10]:
df

,reviewId,content,score
0,1c104f86-7b0e-4efb-8610-7aa2eef5850e,[bagusan],1
1,93849d50-aa43-4bea-8cb4-2b68e70bea03,[confidential],1
2,1520cf87-f7bf-4be6-886a-7c7e6d7d40ac,"[yg, bermutu, jgn, percaya, sma, komen, yg, ko...",1
3,861cd396-7a64-484a-81c2-aae3c986e472,"[menghubungkan, ib, bri]",1
4,bd55e03c-3cf0-4a4c-af68-79bb1883475d,"[perangkat, redmi, note, operasi, android, mem...",1
...,...,...,...
2810,2c62146d-a521-45f9-aaac-5e371f93fc92,"[hallo, tanggal, agustus, poin, pendapatan, se...",1
2811,0bfce725-03e6-4abd-aa1f-7cb68d9a3f6a,"[chat, dapetnya, udah, sombong, user, takabur,...",1
2812,427e2ae7-7fc8-4bf0-bd50-5a3bda4acffe,[],1
2813,45cf797e-27e2-4eac-be22-7c7265caad9c,"[chat, sampek, sore, doang, dpt, poinnya, diki...",1


# Normalization

In [11]:
# from google.colab import drive
# drive.mount('/content/drive/')

# path = 'drive/MyDrive/Colab Notebooks/App Improvement Recommendation System/Data/'

In [12]:
normalizad_word = pd.read_excel(path + 'normalisasi.xlsx')

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

df['content'] = df['content'].apply(normalized_term)

df

,reviewId,content,score
0,1c104f86-7b0e-4efb-8610-7aa2eef5850e,[bagusan],1
1,93849d50-aa43-4bea-8cb4-2b68e70bea03,[confidential],1
2,1520cf87-f7bf-4be6-886a-7c7e6d7d40ac,"[yang, bermutu, jangan, percaya, sma, komen, y...",1
3,861cd396-7a64-484a-81c2-aae3c986e472,"[menghubungkan, ib, bank]",1
4,bd55e03c-3cf0-4a4c-af68-79bb1883475d,"[perangkat, redmi, note, operasi, android, mem...",1
...,...,...,...
2810,2c62146d-a521-45f9-aaac-5e371f93fc92,"[hallo, tanggal, agustus, poin, pendapatan, se...",1
2811,0bfce725-03e6-4abd-aa1f-7cb68d9a3f6a,"[chat, dapetnya, sudah, sombong, user, takabur...",1
2812,427e2ae7-7fc8-4bf0-bd50-5a3bda4acffe,[],1
2813,45cf797e-27e2-4eac-be22-7c7265caad9c,"[chat, sampek, sore, saja, dapat, poinnya, sed...",1


In [13]:
test = df['content'].to_string()
print(type(df['content']))
print(type(test))

<class 'pandas.core.series.Series'>
<class 'str'>


In [14]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
!pip install Swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['content']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))

3708


In [17]:
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])

bagusan : bagus
confidential : confidential
yang : yang
bermutu : mutu
jangan : jangan
percaya : percaya
sma : sma
komen : komen
komena : komena
bntang : bntang
orang : orang
suruhan : suruh
semua : semua
msa : msa
saldo : saldo
hilang : hilang
tanya : tanya
group : group
project : project
lngsung : lngsung
kick : kick
dari : dari
whats : whats
aplikasi : aplikasi
tertawa : tertawa
not : not
recomended : recomended
pakai : pakai
rating : rating
isia : isia
suruh : suruh
like : like
menghubungkan : hubung
ib : ib
bank : bank
perangkat : perangkat
redmi : redmi
note : note
operasi : operasi
android : android
membuka : buka
stuck : stuck
mengecewakan : kecewa
sudah : sudah
tidak : tidak
berberhasil : hasil
menunggu : tunggu
besok : besok
nyabagaimana : nyabagaimana
saja : saja
login : login
kebeberapa : beberapa
perbaiki : baik
bug : bug
integrasinya : integrasi
star : star
for : for
kamu : kamu
respon : respon
email : email
menghilang : hilang
daftr : daftr
riwayat : riwayat
saya : saya


In [18]:
# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['content'] = df['content'].swifter.apply(get_stemmed_term)

print(df['content'])

Pandas Apply:   0%|          | 0/2815 [00:00<?, ?it/s]

0                                                 [bagus]
1                                          [confidential]
2       [yang, mutu, jangan, percaya, sma, komen, yang...
3                                      [hubung, ib, bank]
4       [perangkat, redmi, note, operasi, android, buk...
                              ...                        
2810    [hallo, tanggal, agustus, poin, dapat, sengaja...
2811    [chat, dapetnya, sudah, sombong, user, takabur...
2812                                                   []
2813    [chat, sampek, sore, saja, dapat, poin, sediki...
2814    [kerja, turun, akumulasi, poin, akumulasi, kec...
Name: content, Length: 2815, dtype: object


# Stopword Removal

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend([ 'indonesia', 'kulina', 'aplikasi', 'bobobox', 'oy', 'stockbit', 'payfazz', 'fazz', 'bukuwarung', 'buku', 'warung', 'flip', 'neu', 
                       'happyfresh', 'ipiring', 'nya', 'iya', 'coba', 'sih', 'bintang', 'terima', 'kasih', 'terimakasih', 'the', 'deh', 'in', 'bi',
                       'pagi', 'dong', 'moga', 'ya', 'banget', 'jaya', 'maju', 'bagus', 'suka', 'lumayan', 'cepat', 'tambah', 'tingkat', 'layan', 
                       'ribet', 'oke', 'mantap', 'bantu', 'banget', 'pakai', 'terima', 'kasih', 'keren', 'mudah', 'manfaat', 'kecewa', 'tolong',
                       'gampang', 'sukses', 'admin', 'susah', 'turun', 'naikin', 'auto', 'nomor', 'uninstall', 'hp', 'ganti', 'jelek', 'download',
                       'bangkrut', 'capek', 'melulu', 'ya', 'hilang', 'up', 'masuk', 'hapus', 'lancar', 'pokok', 'alhamdulillah', 'biar', 'minimal', 
                       'laku', 'aman', 'jaga', 'terimakasih', 'terima kasih', 'gagal', 'gratis', 'uang', 'transaksi', 'lari', 'sungguh', 'tanggap',
                       'bikin', 'nyaman', 'sulit', 'otomatis', 'butuh', 'nyata', 'nyangkut', 'selamat', 'nasib', 'kena', 'maaf', 'pindah', 'belah',
                       'maklum', 'kaum', 'pas', 'muncul', 'tulis', 'kontol', 'salah', 'ulang', 'hasil', 'saing', 'kompetitor', 'rilis', 'muat', 'lambat',
                       'kendala', 'pending', 'sesuai', 'nominal', 'eh', 'via', 'tangan', 'sedang apa', 'gunain', 'kali', 'letak', 'ngetik', 'keyboard',
                       'eja', 'buruk', 'matang', 'mulus', 'lebih baik', 'beda', 'ayo', 'tarik', 'tinggal', 'kaya', 'turunin', 'kasi', 'langsung', 'kesini',
                       'share', 'whatsapp', 'galeri', 'paham', 'faktor', 'format', 'mohon', 'kirim', 'efektif', 'nama', 'barang', 'potong', 'angka',
                       'cantik', 'tera', 'jam', 'kadaluarsa', 'proses', 'lapor', 'sistem', 'buka', 'tutup', 'pilih', 'nomor', 'kartu', 'alih', 'utama',
                       'bayar', 'mobile', 'layar', 'mantap', 'mantul', 'terus', 'bot', 'hebat', 'kembang', 'semangat', 'keren', 'rekomemded', 'efisien',
                       'canda', 'zuztjd', 'support', 'digital', 'praktis', 'urus', 'jos', 'gandos', 'pelit', 'jamaah', 'rate', 'malu', 'sampah', 'parah',
                       'setan', 'yh', 'zzz', 'mahal', 'nyesel', 'normal', 'habis', 'kamera', 'nol', 'nom', 'masak', 'si', 'bodoh', 'mentang', 'harap', 'payah',
                       'orang', 'balikin', 'gs', 'rendem', 'bilang', 'nambah', 'manual', 'heuuu', 'henti', 'enak', 'db', 'komenan', 'aa', 'abai', 'heh',
                       'hha', 'dinaikin', 'sekaliuninstal', 'nga', 'zonk', 'hilanggg', 'tolol', 'ngumpulin', 'muter', 'hadehhh', 'nyarinya', 'malas', 'main',
                       'nerima', 'oi', 'asu', 'kendor', 'jancok', 'ora', 'kenek', 'sekrang', 'keluh', 'timbang', 'kucing', 'ku', 'copot', 'indah', 'pasang',
                       'kesel', 'sangat', 'besar', 'isi', 'tunggu'])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['content'] = df['content'].apply(stopwords_removal) 

In [21]:
df

,reviewId,content,score
0,1c104f86-7b0e-4efb-8610-7aa2eef5850e,[],1
1,93849d50-aa43-4bea-8cb4-2b68e70bea03,[confidential],1
2,1520cf87-f7bf-4be6-886a-7c7e6d7d40ac,"[mutu, percaya, sma, komen, komena, bntang, su...",1
3,861cd396-7a64-484a-81c2-aae3c986e472,"[hubung, ib, bank]",1
4,bd55e03c-3cf0-4a4c-af68-79bb1883475d,"[perangkat, redmi, note, operasi, android, stuck]",1
...,...,...,...
2810,2c62146d-a521-45f9-aaac-5e371f93fc92,"[hallo, tanggal, agustus, poin, sengaja, poin]",1
2811,0bfce725-03e6-4abd-aa1f-7cb68d9a3f6a,"[chat, dapetnya, sombong, user, takabur, yakkk]",1
2812,427e2ae7-7fc8-4bf0-bd50-5a3bda4acffe,[],1
2813,45cf797e-27e2-4eac-be22-7c7265caad9c,"[chat, sampek, sore, poin, sedih, pulsa]",1


# Save to Drive

In [22]:
import numpy as np

In [23]:
# df2 = df.replace(r'^\s*$', np.nan, regex=True)
df2 = df[df.astype(str)['content'] != '[]']
df2

,reviewId,content,score
1,93849d50-aa43-4bea-8cb4-2b68e70bea03,[confidential],1
2,1520cf87-f7bf-4be6-886a-7c7e6d7d40ac,"[mutu, percaya, sma, komen, komena, bntang, su...",1
3,861cd396-7a64-484a-81c2-aae3c986e472,"[hubung, ib, bank]",1
4,bd55e03c-3cf0-4a4c-af68-79bb1883475d,"[perangkat, redmi, note, operasi, android, stuck]",1
5,68070afc-8dc6-436a-af63-0e3240a376ff,"[besok, nyabagaimana]",1
...,...,...,...
2809,9d104fe6-e9fc-4289-bca1-ff70d1996680,"[poin, error, top-up]",1
2810,2c62146d-a521-45f9-aaac-5e371f93fc92,"[hallo, tanggal, agustus, poin, sengaja, poin]",1
2811,0bfce725-03e6-4abd-aa1f-7cb68d9a3f6a,"[chat, dapetnya, sombong, user, takabur, yakkk]",1
2813,45cf797e-27e2-4eac-be22-7c7265caad9c,"[chat, sampek, sore, poin, sedih, pulsa]",1


In [24]:
with open(path + 'data-ulasan-oy-preprocessed.csv', 'w') as f:
  df2.to_csv(f)